In [1]:
from trajectree.fock_optics.utils import create_vacuum_state
from trajectree.fock_optics.light_sources import light_source, create_TMSV_OP_Dense
from trajectree.fock_optics.devices import ry, global_phase
from trajectree.fock_optics.measurement import create_threshold_POVM_OP_Dense, rotate_and_measure, create_PNR_POVM_OP_Dense
from trajectree.fock_optics.outputs import print_quantum_state, read_quantum_state
from trajectree.fock_optics.noise_models import general_mixed_bs_noise_model, depolarizing_operators, two_qubit_depolarizing_channel

from trajectree.trajectory import trajectory_evaluator, quantum_channel

import numpy as np
from scipy import sparse as sp
import qutip as qt
from numpy import sqrt
import time
import copy
from quimb.tensor.tensor_arbgeom import tensor_network_apply_op_vec #type: ignore
from quimb.tensor import MatrixProductOperator as mpo #type: ignore

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from functools import lru_cache
from IPython.display import HTML
import multiprocessing


# Parameters

In [2]:
# Physical parameters
N = 4
num_modes = 8
mean_photon_num = 0.06
efficiency = 0.01
depolarizing_error = 0
damping_error = True

# Simulation parameters
cache_size = N
max_cache_nodes = -1
error_tolerance = 1e-10
num_simulations = 1000

verbose = True
progress = [x * num_simulations//10 for x in range(1, 11)]

# Functions

In [3]:
def perform_distribution(psi, signal_angles, idler_angles):
    quantum_channel_list = []
    phase_flip_ops = [sp.csr_matrix(global_phase(0, N, return_unitary = True, site1 = 0, site2 = 1, total_sites = 8))]
    quantum_channel_list.append(quantum_channel(N = N, num_modes = 8, formalism = "kraus", kraus_ops_tuple = ((0,1), phase_flip_ops), name = "first_kraus"))

    PA_quantum_channels, expectation_channels = rotate_and_measure(None, N, psi.site_tags, num_modes, efficiency, error_tolerance, idler_angles, signal_angles, dark_counts = [3e-5,3e-5], rotations = {"signal":(2,3), "idler":(0,1)}, measurements = {1:(0,2), 0:(1,3)}, depolarizing_error = depolarizing_error, damping_error = damping_error, return_quantum_channel = True)

    quantum_channel_list.extend(PA_quantum_channels)

    t_eval = trajectory_evaluator(quantum_channel_list, cache_size = cache_size, max_cache_nodes = max_cache_nodes, calc_expectation = True, observable_ops = expectation_channels)

    probabilities = []
    times = []
    for i in range(num_simulations): 
        start = time.time()
        probabilities.append(t_eval.perform_simulation(psi, error_tolerance, normalize = True))

        time_taken = time.time() - start
        if verbose:
            if i in progress:
                print(f"Completed {progress.index(i)+1}0% of simulations")
        times.append(time_taken)
    return np.mean(probabilities)

# Initial state

In [4]:
vacuum = create_vacuum_state(num_modes=num_modes, N=N)
psi, TMSV_state = light_source(vacuum, N, mean_photon_num, num_modes, error_tolerance, compress=True, contract=True)

idler_angles = [np.pi/2]
signal_angles = [np.pi/2]

p1 = perform_distribution(psi, signal_angles, idler_angles)

Completed 10% of simulations
Completed 20% of simulations
Completed 30% of simulations
Completed 40% of simulations
Completed 50% of simulations
Completed 60% of simulations
Completed 70% of simulations
Completed 80% of simulations
Completed 90% of simulations


In [5]:
vacuum = create_vacuum_state(num_modes=num_modes, N=N)
psi, TMSV_state = light_source(vacuum, N, mean_photon_num, num_modes, error_tolerance, compress=True, contract=True)

idler_angles = [np.pi/2]
signal_angles = [3*np.pi/2]

p2 = perform_distribution(psi, signal_angles, idler_angles)

Completed 10% of simulations
Completed 20% of simulations
Completed 30% of simulations
Completed 40% of simulations
Completed 50% of simulations
Completed 60% of simulations
Completed 70% of simulations
Completed 80% of simulations
Completed 90% of simulations


In [6]:
(p1-p2)/(p1+p2)

np.float64(0.9999999353880786)